In [1]:
#Install required packages

%pip install -U langgraph langchain_community langchain_openai langsmith langgraph-supervisor 

   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------- ----------------------- 1.0/2.5 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 6.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 7.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.58
    Uninstalling langchain-core-0.3.58:
      Successfully uninstalled langchain-core-0.3.58
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.24
    Uninstalling langchain-0.3.24:
      Successfully uninstalled langchain-0.3.24
  Attempting uninstall: langchain_community
    Found existing installation: langchain-community 0.3.23
    Uninstalling langchain-community-0.3.23:
      Successfully uninstalled langchain-community-0.3.23
  Attempting uninstall: langgraph-supervisor
    Found existing installat

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langgraph-swarm 0.0.10 requires langgraph<0.4.0,>=0.3.5, but you have langgraph 0.4.3 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Environment Variable Initialization

import getpass
import os

def _set_if_undefined(var_name: str):
    """
    Set an environment variable if it is not already defined.
    
    Args:
        var_name (str): Name of the environment variable to set.
    """
    if not os.environ.get(var_name):
        # Securely prompt the user for input without echoing it on screen
        os.environ[var_name] = getpass.getpass(f"Please provide your {var_name}: ")

# ---- Environment Variables Required ----

_set_if_undefined("OPENAI_API_KEY")         # API key for OpenAI models
_set_if_undefined("LANGSMITH_TRACING")      # Enable LangSmith tracing ("true" to enable)
_set_if_undefined("LANGSMITH_API_KEY")      # API key for LangSmith platform
_set_if_undefined("MODEL")                  # Model name (e.g., "gpt-4.1" "gpt-4o", "gpt-3.5-turbo")

In [ ]:
# Multi-Agent Orchestration with LangGraph:
#- Supervisor agent coordinates between specialized workers.
#- Workers: weather reporting agent, dressing planner agent.

# ---- Imports ----

import os
from langchain_openai import ChatOpenAI
from typing import Annotated
from langgraph_supervisor import create_supervisor
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

# ---- LLM Setup ----

# Load the default model from environment variables
default_model = os.environ["MODEL"]

# Initialize the LLM (Large Language Model) interface
llm = ChatOpenAI(model=default_model)

# ---- Node Definitions ----
@tool
def weather_reporting_tool(city: Annotated[str, "name of the city"]):
    """Tool to fetch current weather data for a given city."""
     # (Simulated API call — replace with real API integration)
    json_data = {
        "weather": {
            "name": city,
            "main": "Rain",
            "description": "moderate rain",
            "temp": 284.2,
            "feels_like": 282.93,
            "temp_min": 283.06,
            "temp_max": 286.82,
            "pressure": 1021,
            "humidity": 60,
            "sea_level": 1021,
            "grnd_level": 910
        }
    }
    return json_data
    

# Weather Reporting Agent
weather_reporting_agent = create_react_agent(
    llm,
    tools=[weather_reporting_tool],
    name="weather_reporting_agent",
    prompt=(
        "You are a weather reporter. Report current weather for the provided city. "
        "You may use tools. Do not suggest what to wear."
    )
)

# Dressing Planner Agent
dressing_planner_agent = create_react_agent(
    llm,
    tools=[],
    name="dressing_planner_agent",
    prompt=(
        "You suggest dressing options based on the current weather. "
        "Prioritize 'feels like' temperature and consider wind conditions."
    )
)

# ---- Supervisor Setup ----

# System prompt guiding the supervisor's behavior
system_prompt = (
    "# Role and Objective"
    "You are a Supervisor Agent tasked with managing a conversation between two specialized workers: "
    "`weather_reporting_agent` and `dressing_planner_agent`."
    "Your goal is to orchestrate their actions to resolve the user's request efficiently."
    "# Instructions"
    " - Persist through multiple steps until the task is fully complete."
    " - Always select the next worker based on context."
    " - Think step-by-step before choosing a worker and after receiving results."
    "# Reasoning Steps"
    "1. Analyze current state."
    "2. Plan the next best action."
    "3. Reflect after worker output."
    "4. Repeat until completion."
    "# Tool/Worker Use"
    "- `weather_reporting_agent`: gather or analyze weather."
    "- `dressing_planner_agent`: suggest clothing based on weather."
)

# Create supervisor 
builder = create_supervisor(
    agents=[weather_reporting_agent, dressing_planner_agent],
    model=llm,
    output_mode="last_message",
    prompt=(system_prompt)
)


  

In [15]:
# Compile the graph
graph = builder.compile()


In [16]:
# ---- Simulation ----

for s in graph.stream(
    {"messages": [("user", "Stockholm")]}, debug=True):
    print(s)
    print("============================")


[-1:checkpoint] State at the end of step -1:
{'messages': []}
[0:tasks] Starting 1 task for step 0:
- __start__ -> {'messages': [('user', 'Stockholm')]}
[0:writes] Finished step 0 with writes to 1 channel:
- messages -> [('user', 'Stockholm')]
[0:checkpoint] State at the end of step 0:
{'messages': [HumanMessage(content='Stockholm', additional_kwargs={}, response_metadata={}, id='37d27549-55f0-4577-bed8-dc9282f6f57c')]}
[1:tasks] Starting 1 task for step 1:
- supervisor -> {'is_last_step': False,
 'messages': [HumanMessage(content='Stockholm', additional_kwargs={}, response_metadata={}, id='37d27549-55f0-4577-bed8-dc9282f6f57c')],
 'remaining_steps': 24}
[1:writes] Finished step 1 with writes to 1 channel:
- messages -> [HumanMessage(content='Stockholm', additional_kwargs={}, response_metadata={}, id='37d27549-55f0-4577-bed8-dc9282f6f57c'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_svc6kIdpHTsDTHCCtdwfhpZp', 'function': {'arguments': '{}', 'name': 'transfer_